## BigData Assignment-2

### Import the relevant packages

In [14]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [15]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [16]:
session.execute("DROP KEYSPACE IF EXISTS m14")

### Iterate through the results and print the names of the keyspaces.


In [17]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [18]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [19]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Creating the inventory table

In [20]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    sku VARCHAR, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(sku,name)
);
""")

### Load the data

In [21]:
df = pd.read_csv('m14.csv')
df

,sku,name,description,warehouse_num
0,'UGG-BB-PUR-01','Food','This is the foremost important thing',123
1,'UGG-BB-PUR-02','Accessories','This needs to be stocked',123
2,'UGG-BB-PUR-03','Electronics','More of the new stock is coming',212
3,'UGG-BB-PUR-04','Cosmetics','Need to be handled with care',345
4,'UGG-BB-PUR-05','Stationary','Does not accomodate much space',123
5,'UGG-BB-PUR-06','Furniture','Accomodates more space',432
6,'UGG-BB-PUR-07','Perfumes','This needs more packaging',212


In [22]:
for index, row in df.iterrows():
    print(f"sku = {row.sku}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}");

sku = 'UGG-BB-PUR-01', name =0, description = 'This is the foremost important thing', warehouse_num = 123
sku = 'UGG-BB-PUR-02', name =1, description = 'This needs to be stocked', warehouse_num = 123
sku = 'UGG-BB-PUR-03', name =2, description = 'More of the new stock is coming', warehouse_num = 212
sku = 'UGG-BB-PUR-04', name =3, description = 'Need to be handled with care', warehouse_num = 345
sku = 'UGG-BB-PUR-05', name =4, description = 'Does not accomodate much space', warehouse_num = 123
sku = 'UGG-BB-PUR-06', name =5, description = 'Accomodates more space', warehouse_num = 432
sku = 'UGG-BB-PUR-07', name =6, description = 'This needs more packaging', warehouse_num = 212


### Insert the values into the table

In [23]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('UGG-BB-PUR-01', 'Food', 'This is the foremost important thing', 123);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('UGG-BB-PUR-02', 'Accessories', 'This needs to be stocked', 123);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('UGG-BB-PUR-03', 'Electronics', 'More of the new stock is coming', 212);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('UGG-BB-PUR-04', 'Cosmetics', 'Need to be handled with care', 345);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('UGG-BB-PUR-05', 'Stationary', 'Does not accomodate much space', 123);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('UGG-BB-PUR-06', 'Furniture', 

### Executing the below query without creating Index variable

In [24]:
rows = session.execute("select name from m14.inventory where warehouse_num = 123")
for row in rows:
    print(f"{row[0]}")

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

### Creating the index variable on the warehouse_num column

In [ ]:
session.execute(f"""
        CREATE INDEX IF NOT EXISTS warehouse
ON m14.inventory ( warehouse_num );
        """)


In [ ]:
rows = session.execute("select name from m14.inventory where warehouse_num = 123")
for row in rows:
    print(f"{row[0]}")

Accessories
Food
Stationary
